### Vector stores and retrievers
We will familiarize you with LangChain's vector store and retriever abstractions. These abstractions are designed to support retrieval of data-- from (vector) databases and other sources-- for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation.

We will cover 
- Documents
- Vector stores
- Retrievers


### Documents
LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

- page_content: a string representing the content;
- metadata: a dict containing arbitrary metadata.
The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

Let's generate some sample documents:

In [2]:
from langchain_core.documents import Document

In [3]:
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

Create a chat model instance from Groq API

In [5]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')

### Access the Llama3-8b-8192 model --> Supported models list is here https://console.groq.com/docs/models 
llm = ChatGroq(model='Llama3-8b-8192' , groq_api_key = groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023B004A2CD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023B0095E610>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

### Embeddings

We have to convert the documents in embeddings. For this we have to use the embedding models. We are using HuggingFace Embedding models that are open sourcey. We have to install langchain-huggingface library

There are many embedding models out of which we are using the 'all-MiniLM-L6-v2'

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

The model gets downloaded locally on your windows machine at 'C:\Users\Admin\.cache\huggingface\hub'

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

### Vector Store

Documentation -->

1) https://python.langchain.com/docs/concepts/vectorstores/

2) https://python.langchain.com/docs/concepts/vectorstores/#interface

We will be using Chroma Vector Store in memory implementation. You can also save the db locally

In [8]:
from langchain_chroma import Chroma

In [9]:
vector_store = Chroma.from_documents(documents=documents , embedding=embeddings)
vector_store

Performing similarity search on Vector Store

In [10]:
vector_store.similarity_search("cat")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

Async Operations


Vector stores are usually run as a separate service that requires some IO operations, and therefore they might be called asynchronously. That gives performance benefits as you don't waste time waiting for responses from external services. That might also be important if you work with an asynchronous framework, such as FastAPI.


LangChain supports async operation on vector stores. All the methods might be called using their async counterparts, with the prefix a, meaning async.


Documentation --> https://python.langchain.com/docs/how_to/vectorstores/



In [11]:
await vector_store.asimilarity_search("cat")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

If we want to get the score of similarity search we can use the function 'similarity_search_with_score'


In [12]:
vector_store.similarity_search_with_score('cat')

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351057410240173),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740898847579956),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.5956902503967285),
 (Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.6657923460006714)]

### Retrievers

Documentation - 

https://python.langchain.com/docs/concepts/retrievers/



LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method:

Method 1 - Creating a Retriever

Below is one of the way of creating a retriever

In [13]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever=RunnableLambda(vector_store.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

Method 2 

Vectorstores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:

In [14]:
retriever=vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)
retriever.batch(["cat","dog"])


[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

### Implementing a RAG pipeline

In [15]:
### Step 1 - Create a prompt template

from langchain_core.prompts import ChatPromptTemplate

In [16]:
message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""


prompt = ChatPromptTemplate.from_messages([('human' , message)])

In [17]:
### Step 2 - Create a RAG chain 

### context_from_retriever | prompt | llm

from langchain_core.runnables import RunnablePassthrough

context_from_retriever = {"context":retriever,"question":RunnablePassthrough()}

rag_chain = context_from_retriever | prompt | llm

In [19]:
response = rag_chain.invoke('tell me about dogs')
response.content

'According to the provided context, dogs are great companions, known for their loyalty and friendliness.'